**Initialization**

In [210]:
import google.generativeai as genai
import json
import os
from dotenv import load_dotenv
import easyocr
import cv2
from rapidfuzz import fuzz

load_dotenv()

True

**Gemini Model**

In [211]:
def generate(prompt, image_path) -> list | dict:
    api_key = os.getenv("GEMINI_API_KEY")
    genai.configure(api_key=api_key)

    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash",
        system_instruction="You are a helpful assistant that extracts newspaper fields from images.",
        generation_config={"response_mime_type": "application/json"}
    )

    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()

    response = model.generate_content([
        {"text": prompt},
        {"mime_type": "image/png", "data": image_bytes}
    ])

    raw_json = response.text
    data = json.loads(raw_json)

    return data

**Call to generate from Gemini**

In [212]:
headline_schema = {
	"type": "array",
	"items": {"type": "string"}
}

headline_prompt = (
	"You are given a newspaper image. "
	"Extract only the article all possible headlines — ignore advertisements, captions, subheadlines, and any other text. "
	"Return the result strictly matching this JSON schema:\n\n"
	f"{json.dumps(headline_schema, indent=2)}"
)

target_image_path = "page_8.png"

headlines = generate(headline_prompt, target_image_path)

for i, headline in enumerate(headlines):
    headlines[i] = headline
print(headlines)
# Dev log
print(json.dumps(headlines, indent=2, ensure_ascii=False))

["Thunderbelles' 5th-set grit turns back Crossovers", "Tenorio to follow in Jawo's footsteps as playing coach?", 'Caloocan, Pangasinan hurdle rivals', 'Cone lauds CJ Perez for helping Gilas defeat Black Bears', 'Edoc shines as weather whips up surprises at JPGT Riviera']
[
  "Thunderbelles' 5th-set grit turns back Crossovers",
  "Tenorio to follow in Jawo's footsteps as playing coach?",
  "Caloocan, Pangasinan hurdle rivals",
  "Cone lauds CJ Perez for helping Gilas defeat Black Bears",
  "Edoc shines as weather whips up surprises at JPGT Riviera"
]


**EasyOCR reads**

In [213]:
reader = easyocr.Reader(['en'])
image = cv2.imread(target_image_path)
results = reader.readtext(image)

image_raw = image.copy()
for (top_left, top_right, bottom_right, bottom_left), text, confidence in results:
    tl = (int(top_left[0]), int(top_left[1]))
    br = (int(bottom_right[0]), int(bottom_right[1]))
    cv2.rectangle(image_raw, tl, br, (0, 0, 255), 2)
    coord_label = f"{tl} {br}"
    cv2.putText(image_raw, coord_label, (tl[0], tl[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

output_path_raw = target_image_path.replace(".png", "_ocr_boxes.png")
cv2.imwrite(output_path_raw, image_raw)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


True

**Finds the coordinates of the headings**

In [214]:
bounding_box_text = []
bounding_box_coordinates = []

for coordinates, text, _ in results:
    print(text)
    for headline in headlines:
        
        score = fuzz.partial_ratio(headline, text)

        if score > 80 and len(text) > 3:

            top_left, _, bottom_right, _ = coordinates
            
            current_box = ((int(top_left[0]), int(top_left[1])),
                           (int(bottom_right[0]), int(bottom_right[1])))
            bounding_box_text.append(text)
            bounding_box_coordinates.append(current_box)

print(bounding_box_text)
print(bounding_box_coordinates)
print(len(bounding_box_text), len(bounding_box_coordinates))

for i in range(len(bounding_box_text)):
    
    print(bounding_box_text[i])
    print(bounding_box_coordinates[i])


A8
SPORTS
WEDNESDAY, JULY 30, 2025
RIERA U: MALLARI; Editor
RANDY M: CALUAG, Asst. Editor
EDGARD HILARIO, Asst. Editor
standard
The ZUS
G
Coffee
Thunderbelles
celebrate
their conquest
Thunderbelles'
24
cfere Tiggo
Idun
Crossovers in
Sth-set grit
the PVL On
Tour Tuesday
at the Candon
turns back
City Arena.
Crossovers
CANDON, Ilocos Sur
Given the op
portunity
to rise
ZUS Coffee seized
the moment in
grueling, high-stakes
battle the Thunderbelles seemed tailor-
made for; delivering
poised yet fear-
less performance
to shock
stunned
Tiggo side, 19-25,.25-22,
18-25,
25-19, 15-6,in the PVL On Tour Tues-
at the Candon City Arena_
Down   1-2 in
sets   and
teetering   on
the brink;, the Thunderbelles mounted
blistering
turnaround  midway   through
the fourth before
dominating the decider
in stunning
runawav
fashion
mo
ments   earlier; the Crossovers  appeared
headed for an
outright quarterfinals berth
only to watch their composure unravel
in the face ofZUS Coffee s signature grit
and go-for-brok

In [215]:
def is_close(coordinate1, coordinate2, gap_x=5, gap_y=5):
    (x1_min, y1_min), (x1_max, y1_max) = coordinate1
    (x2_min, y2_min), (x2_max, y2_max) = coordinate2

    # Expand both boxes by the gap
    x1_min, y1_min, x1_max, y1_max = x1_min - gap_x, y1_min - gap_y, x1_max + gap_x, y1_max + gap_y
    x2_min, y2_min, x2_max, y2_max = x2_min - gap_x, y2_min - gap_y, x2_max + gap_x, y2_max + gap_y

    # Overlap condition (after expansion)
    horizontal_overlap = not (x1_max < x2_min or x2_max < x1_min)
    vertical_overlap = not (y1_max < y2_min or y2_max < y1_min)

    return horizontal_overlap and vertical_overlap


new = {}
print("Headlines:", headlines)
for headline in headlines:
    new[headline] = []

for i, text in enumerate(bounding_box_text):
    box = bounding_box_coordinates[i]
    for headline in new:
        score = fuzz.partial_ratio(headline, text)
        if score > 80 and len(text) > 3:
            if not new[headline]:
                new[headline].append({"text": text, "box": box})
                print("new")
                print(f"{text=}, {box=}")
            else:
                for i, currentBox in enumerate(new[headline]):
                    
                    if is_close(currentBox["box"], box):
                        print(f"{text=}, {box=}, {currentBox=}")
                        (ex_tl_x, ex_tl_y), (ex_br_x, ex_br_y) = currentBox["box"]
                        (tl_x, tl_y), (br_x, br_y) = box

                        new_tl = (min(ex_tl_x, tl_x), min(ex_tl_y, tl_y))
                        new_br = (max(ex_br_x, br_x), max(ex_br_y, br_y))

                        new[headline][i]["box"] = (new_tl, new_br)
                        new[headline][i]["text"] += " " + text
                        break
                else:
                    new[headline].append({"text": text, "box": box})

print(json.dumps(new, indent=2, ensure_ascii=False))

Headlines: ["Thunderbelles' 5th-set grit turns back Crossovers", "Tenorio to follow in Jawo's footsteps as playing coach?", 'Caloocan, Pangasinan hurdle rivals', 'Cone lauds CJ Perez for helping Gilas defeat Black Bears', 'Edoc shines as weather whips up surprises at JPGT Riviera']
new
text='Thunderbelles', box=((3335, 803), (3594, 846))
text='Sth-set grit', box=((107, 1035), (652, 1205)), currentBox={'text': "Thunderbelles'", 'box': ((103, 888), (798, 1046))}
text='turns back', box=((98, 1174), (607, 1319)), currentBox={'text': "Thunderbelles' Sth-set grit", 'box': ((103, 888), (798, 1205))}
text='Crossovers', box=((105, 1312), (596, 1450)), currentBox={'text': "Thunderbelles' Sth-set grit turns back", 'box': ((98, 888), (798, 1319))}
new
text='Tenorio to follow in Jawos', box=((816, 2240), (3465, 2511))
text='footsteps as playing coach?', box=((794, 2482), (3587, 2862)), currentBox={'text': 'Tenorio to follow in Jawos', 'box': ((816, 2240), (3465, 2511))}
new
text='Cone lauds CJ Pere

In [216]:
for key in new:
    print(new[key])
    for i in range(len(new[key])):
        for j in range(i + 1, len(new[key])):
            if is_close(new[key][i]["box"], new[key][j]["box"]):
                print(f"Merging {new[key][i]} and {new[key][j]}")
                (ex_tl_x, ex_tl_y), (ex_br_x, ex_br_y) = new[key][i]["box"]
                (tl_x, tl_y), (br_x, br_y) = new[key][j]["box"]

                new_tl = (min(ex_tl_x, tl_x), min(ex_tl_y, tl_y))
                new_br = (max(ex_br_x, br_x), max(ex_br_y, br_y))

                new[key][i]["box"] = (new_tl, new_br)
                new[key][i]["text"] += " " + new[key][j]["text"]
                del new[key][j]
                break
print(new)

[{'text': 'Thunderbelles', 'box': ((3335, 803), (3594, 846))}, {'text': "Thunderbelles' Sth-set grit turns back Crossovers", 'box': ((98, 888), (798, 1450))}, {'text': 'Crossovers in', 'box': ((3335, 1030), (3573, 1073))}, {'text': 'over', 'box': ((202, 2828), (282, 2866))}]
[{'text': 'Tenorio to follow in Jawos footsteps as playing coach?', 'box': ((794, 2240), (3587, 2862))}, {'text': 'Tenorio', 'box': ((2348, 3166), (2476, 3203))}, {'text': 'Tenorio', 'box': ((1932, 3430), (2075, 3472))}, {'text': 'following', 'box': ((1785, 6023), (1959, 6086))}]
[{'text': 'Caloocan; Pangasinan hurdle rivals', 'box': ((100, 3999), (638, 4409))}, {'text': 'Caloocan', 'box': ((247, 4681), (416, 4731))}, {'text': 'Pangasinan', 'box': ((442, 4857), (643, 4914))}, {'text': 'Caloocan;', 'box': ((559, 5251), (753, 5299))}]
[{'text': 'Cone lauds CJ Perez for helping Gilas defeat Black Bears Gilas', 'box': ((826, 3844), (3534, 4089))}, {'text': 'helping', 'box': ((3227, 4453), (3353, 4514))}]
[{'text': 'Edo

In [217]:
from fuzzywuzzy import process

image_merged = image.copy()

for key in new:
    query = key
    choices = [i["text"] for i in new[key]]

    if not choices:
        continue  # skip if no choices

    # map choice → index
    choices_dict = {c: i for i, c in enumerate(choices)}

    # get best match
    best_match = process.extractOne(query, list(choices_dict.keys()))
    if best_match:
        text, score = best_match
        index = choices_dict[text]

        tl, br = new[key][index]["box"]
        print(text, tl, br)
        cv2.rectangle(image_merged, tl, br, (0, 255, 0), 5)

# save output
output_path_merged = f"{os.path.splitext(target_image_path)[0]}_result{os.path.splitext(target_image_path)[1]}"
cv2.imwrite(output_path_merged, image_merged)


Thunderbelles' Sth-set grit turns back Crossovers (98, 888) (798, 1450)
Tenorio to follow in Jawos footsteps as playing coach? (794, 2240) (3587, 2862)
Caloocan; Pangasinan hurdle rivals (100, 3999) (638, 4409)
Cone lauds CJ Perez for helping Gilas defeat Black Bears Gilas (826, 3844) (3534, 4089)
Edoc shines as weather whips up surprises at JPGT Riviera (1039, 5301) (2493, 5619)


True